In [ ]:
import pandas as pd

df = pd.read_csv("all_sets_df.csv")
gene_df = df[df["type"] == "gene"]

df

In [ ]:
METADATA_COLS = ["type", "row", "col", "well", "set", "type.f", "slide"]

# Get feature columns (all columns except metadata)
FEATURE_COLS = [col for col in df.columns if col not in METADATA_COLS]

TREATMENTS = set(["gene"])
CONTROLS = set(df["type"]) - TREATMENTS
print(f"Unique types: {set(df['type'])}")  # Unique types
print(f"Treatments: {TREATMENTS}")
print(f"Controls: {CONTROLS}")

In [ ]:
from scipy.spatial import distance
import numpy as np


# Extract the data for wells with type 'rf' (negative control)
gene_data = gene_df[FEATURE_COLS].values

# Calculate the mean vector and covariance matrix of the reference data
gene_mean = np.mean(gene_data, axis=0)
gene_cov = np.cov(gene_data, rowvar=False)


# Function to calculate Mahalanobis distance
def calculate_mahalanobis(x, mean, cov):
    """Calculate the Mahalanobis distance between x and the distribution defined by mean and cov."""
    return distance.mahalanobis(x, mean, np.linalg.inv(cov))


# Calculate Mahalanobis distance for each row in the dataframe
print("Calculating Mahalanobis distance for each sample...")
df["mahalanobis_distance"] = df.apply(
    lambda row: calculate_mahalanobis(row[FEATURE_COLS], gene_mean, gene_cov), axis=1
)

# Calculate the 95th percentile threshold using only the gene data
gene_distances = df[df["type"] == "gene"]["mahalanobis_distance"]
threshold = np.percentile(gene_distances, 100 - (100 / 3520) * 100)

# Create a new column indicating whether each sample exceeds the threshold
df["mahalanobis_outlier"] = df["mahalanobis_distance"] > threshold

# Count how many samples of each type exceed the threshold
threshold_summary = df.groupby("type")["mahalanobis_outlier"].agg(["count", "sum"])
threshold_summary["percentage"] = (
    threshold_summary["sum"] / threshold_summary["count"]
) * 100
print(threshold_summary)

In [ ]:
from sklearn.ensemble import IsolationForest


clf = IsolationForest(random_state=0, contamination=100 / 3520).fit(
    gene_df[FEATURE_COLS].values
)
outlier_scores = clf.predict(df[FEATURE_COLS].values)

# outlier_scores = IsolationForest(
#     random_state=0, contamination=((176) * 2 + 100) / 3520
# ).fit_predict(df[FEATURE_COLS].values)

df["isolation_forest_outlier"] = outlier_scores == -1

# Count how many samples of each type exceed the threshold
threshold_summary = df.groupby("type")["isolation_forest_outlier"].agg(["count", "sum"])
threshold_summary["percentage"] = (
    threshold_summary["sum"] / threshold_summary["count"]
) * 100
print(threshold_summary)


In [ ]:
import sklearn.neighbors


# Detect outliers using LocalOutlierFactor
outlier_scores = sklearn.neighbors.LocalOutlierFactor(contamination=0.15).fit_predict(
    df[FEATURE_COLS].values
)

df["lof_outlier"] = outlier_scores == -1

# Count how many samples of each type exceed the threshold
threshold_summary = df.groupby("type")["lof_outlier"].agg(["count", "sum"])
threshold_summary["percentage"] = (
    threshold_summary["sum"] / threshold_summary["count"]
) * 100
print(threshold_summary)


In [ ]:
df

In [7]:
import plotly.graph_objects as go


def plot_3d_scatter(
    df,
    x_col,
    y_col,
    z_col,
    type_col="type",
    outlier_col=None,
    title="3D Scatter Plot",
    hover_data=None,
):
    """
    Create a 3D scatter plot colored by type using customdata for hover info.

    Parameters:
    -----------
    df : pandas DataFrame
        The data to plot
    x_col, y_col, z_col : str
        Column names for the x, y, and z axes
    type_col : str, default="type"
        Column name for the categorical variable to color by
    title : str, default="3D Scatter Plot"
        Title for the plot
    hover_data : list, default=None
        List of column names to include in hover information (e.g., ["row", "col", "well"])
    """
    fig = go.Figure()

    # Default hover data if none provided
    if hover_data is None:
        hover_data = []

    # Always include type_col in hover_data if not already present
    if type_col not in hover_data:
        hover_data = [type_col] + hover_data

    # Construct hovertemplate string dynamically
    ht = ""
    for i, col_name in enumerate(hover_data):
        # Use the corresponding customdata index for each column name
        ht += f"<b>{col_name}</b>: %{{customdata[{i}]}}<br>"
    ht += "<extra></extra>"  # Hide the default trace info box like 'trace 0'

    # Get a consistent color for this type_name
    # Get the default plotly colors
    import plotly.express as px

    # Create a mapping from type to color using Dark2 and Set2 color palettes
    type_names = df[type_col].unique()
    dark2_colors = px.colors.qualitative.Dark2  # For outliers
    set2_colors = px.colors.qualitative.Set2  # For non-outliers

    # Create two color mappings for each type - one for outliers and one for non-outliers
    type_colors = {}
    type_colors_outlier = {}

    for i, t in enumerate(type_names):
        dark2_idx = i % len(dark2_colors)
        set2_idx = i % len(set2_colors)
        type_colors[t] = set2_colors[set2_idx]  # Regular points
        type_colors_outlier[t] = dark2_colors[dark2_idx]  # Outlier points

    # Plot points for each type with different colors
    for type_name in df[type_col].unique():
        if outlier_col is not None:
            # Create separate traces for outliers and non-outliers

            for is_outlier in [True, False]:
                mask = (df[type_col] == type_name) & (df[outlier_col] == is_outlier)
                if mask.any():  # Only create trace if there are points
                    df_filtered = df[mask]
                    customdata_values = df_filtered[hover_data].values

                    name = f"{type_name} (Outlier)" if is_outlier else str(type_name)

                    trace = go.Scatter3d(
                        x=df_filtered[x_col],
                        y=df_filtered[y_col],
                        z=df_filtered[z_col],
                        mode="markers",
                        name=name,
                        marker=dict(
                            size=6,
                            opacity=1 if is_outlier else 0.8,
                            symbol="diamond" if is_outlier else "circle",
                            line=dict(
                                width=1 if is_outlier else 0,
                                color="black",  # Keep border color consistent
                            ),
                            color=type_colors_outlier[type_name]
                            if is_outlier
                            else type_colors[type_name],
                        ),
                        legendgroup=name,  # Group traces by type_name
                        customdata=customdata_values,
                        hovertemplate=ht,
                    )

                    fig.add_trace(trace)
        else:
            # Just create traces by type
            mask = df[type_col] == type_name
            df_filtered = df[mask]
            customdata_values = df_filtered[hover_data].values

            trace = go.Scatter3d(
                x=df_filtered[x_col],
                y=df_filtered[y_col],
                z=df_filtered[z_col],
                mode="markers",
                name=str(type_name),  # Ensure name is a string
                marker=dict(size=6, opacity=0.7),
                customdata=customdata_values,
                hovertemplate=ht,
            )

            fig.add_trace(trace)

    # Update layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title=x_col,
            yaxis_title=y_col,
            zaxis_title=z_col,
        ),
        width=1200,
        height=1200,
        showlegend=True,
    )
    fig.show()
    return fig


def save_plot(fig, filename):
    """Save the plot as HTML."""
    fig.write_html(filename)

In [8]:
import pandas as pd


def create_dimension_reduction_df(
    transformed_data,
    n_components,
    prefix="UMAP",
    original_df=None,
    type_col="type",
    hover_data=None,
):
    """Create a DataFrame from UMAP results with optional metadata from original DataFrame."""
    transformed_data = transformed_data[:, :n_components]
    columns = [f"{prefix}{i + 1}" for i in range(n_components)]
    result_df = pd.DataFrame(transformed_data, columns=columns)

    if original_df is not None:
        # Add type column if it exists
        if type_col in original_df.columns:
            result_df[type_col] = original_df[type_col]
        else:
            raise ValueError(f"type_col '{type_col}' not found in original DataFrame.")

        # Add hover data columns if specified
        if hover_data is not None:
            for col in hover_data:
                if col in original_df.columns:
                    result_df[col] = original_df[col]
                else:
                    raise ValueError(
                        f"hover_data column '{col}' not found in original DataFrame."
                    )

    return result_df

In [9]:
from sklearn.preprocessing import StandardScaler


def preprocess_data(df, feature_cols):
    """Standardize the feature data."""
    X = df[feature_cols]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X, X_scaled


X, X_scaled = preprocess_data(df, FEATURE_COLS)

In [ ]:
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import numpy as np


def perform_pca(X_scaled):
    """Perform PCA and return transformed data and PCA object."""
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    return X_pca, pca


def plot_explained_variance(pca):
    """Plot the cumulative explained variance ratio."""
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

    plt.figure(figsize=(10, 6))
    plt.plot(
        range(1, len(explained_variance_ratio) + 1), cumulative_variance_ratio, "bo-"
    )
    plt.xlabel("Number of Components")
    plt.ylabel("Cumulative Explained Variance Ratio")
    plt.title("PCA Explained Variance Ratio")
    plt.grid(True)
    plt.show()

    return cumulative_variance_ratio


HOVER_COLS = [
    "row",
    "col",
    "well",
    "set",
    "mahalanobis_outlier",
    "isolation_forest_outlier",
]

X_pca, pca = perform_pca(X_scaled)
plot_explained_variance(pca)


# Create PCA DataFrame and plot
pca_df = create_dimension_reduction_df(
    X_pca, 3, "PC", df, type_col="type", hover_data=HOVER_COLS
)

pca_fig = plot_3d_scatter(
    pca_df,
    "PC1",
    "PC2",
    "PC3",
    title="3D PCA Projection",
    hover_data=HOVER_COLS,  # Pass the list of *additional* columns to show
    type_col="type",  # Specify the column used for coloring/grouping
    outlier_col="isolation_forest_outlier",
)

save_plot(pca_fig, "pca_3d_plot.html")


In [ ]:
# Apply UMAP for dimensionality reduction

from umap import UMAP


def perform_umap(data, n_components=3, random_state=42, n_neighbors=100, min_dist=0.8):
    """Perform UMAP dimensionality reduction on the input data."""
    umap_model = UMAP(
        n_components=n_components,
        random_state=random_state,
        n_neighbors=n_neighbors,
        min_dist=min_dist,
    )
    umap_result = umap_model.fit_transform(data)
    return umap_result, umap_model


# Perform UMAP and create DataFrame
X_umap, umap_model = perform_umap(X_scaled)
umap_df = create_dimension_reduction_df(X_umap, 3, "UMAP", df, hover_data=HOVER_COLS)

# Plot UMAP results
umap_fig = plot_3d_scatter(
    umap_df,
    "UMAP1",
    "UMAP2",
    "UMAP3",
    title="3D UMAP Projection",
    type_col="type",
    hover_data=HOVER_COLS,
    outlier_col="isolation_forest_outlier",
)

save_plot(umap_fig, "umap_3d_plot.html")
